In [54]:
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import pandas as pd
import numpy as np
from scipy.stats import pointbiserialr, spearmanr

In [55]:
data = pd.read_csv('C:/Users/Lenovo/Desktop/data/adult12.csv')

In [56]:
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [57]:
n_records= data.shape[0]
n_greater_50k = data[data['income'] == '>50k'].shape[0]
n_at_most_50k = data[data['income'] == '<=50k'].shape[0]
greater_percent = (n_greater_50k / n_records) * 100
print("Total number of records: {}".format(n_greater_50k))
print("Individuals making more than $50,000: {}".format(n_greater_50k))
print("Individuals making at most $50,000: {}".format(n_at_most_50k))
print("Percentage of individuals making more than $50,000: {}%".format(greater_percent))

Total number of records: 0
Individuals making more than $50,000: 0
Individuals making at most $50,000: 0
Percentage of individuals making more than $50,000: 0.0%


In [58]:
col_names = data.columns
num_data = data.shape[0]
for c in col_names:
    num_non = data[c].isin(["?"]).sum()
    if num_non >0:
        print (c)
        print (num_non)
        print ("{0:.2f}%".format(float(num_non) / num_data*100))

In [59]:
data=data[data["workclass"] != "?"]
data=data[data["occupation"] != "?"]
data=data[data["native-country"] != "?"]

In [60]:
data.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [61]:
category_col =['workclass','race','education','marital-status','occupation','relationship','sex','native-country','income']
for c in category_col:
    print (c)
    print (data[c].value_counts())

workclass
 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64
race
 White                 27816
 Black                  3124
 Asian-Pac-Islander     1039
 Amer-Indian-Eskimo      311
 Other                   271
Name: race, dtype: int64
education
 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64
marital-status
 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated             

In [62]:
data["income"].value_counts()[0]/data.shape[0]

0.7591904425539756

In [63]:
data["income"].value_counts()[1]/data.shape[0]

0.2408095574460244

In [64]:
income_raw =data['income']
features_raw = data.drop('income', axis=1)

In [65]:
skewed=['capital-gain','capital-loss']
features_log_transformed = pd.DataFrame(data= features_raw)
features_log_transformed[skewed] = features_raw[skewed].apply(lambda x: np.log(x+1))

In [66]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
numerical=['age','education-num','capital-gain','capital-loss','hours-per-week']
features_log_minmax_transform = pd.DataFrame(data = features_log_transformed)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_log_transformed[numerical])
display(features_log_minmax_transform.head(n = 5))

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,0.301370,State-gov,77516,Bachelors,0.800000,Never-married,Adm-clerical,Not-in-family,White,Male,0.667492,0.0,0.397959,United-States
1,0.452055,Self-emp-not-inc,83311,Bachelors,0.800000,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.000000,0.0,0.122449,United-States
2,0.287671,Private,215646,HS-grad,0.533333,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.000000,0.0,0.397959,United-States
3,0.493151,Private,234721,11th,0.400000,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.000000,0.0,0.397959,United-States
4,0.150685,Private,338409,Bachelors,0.800000,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.000000,0.0,0.397959,Cuba


In [67]:
features_final = pd.get_dummies(features_log_minmax_transform)
income = income_raw.map({'<=50K' :0,'>50K' :1})
encoded =list(features_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded)))
encoded

108 total features after one-hot encoding.


['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'workclass_ ?',
 'workclass_ Federal-gov',
 'workclass_ Local-gov',
 'workclass_ Never-worked',
 'workclass_ Private',
 'workclass_ Self-emp-inc',
 'workclass_ Self-emp-not-inc',
 'workclass_ State-gov',
 'workclass_ Without-pay',
 'education_ 10th',
 'education_ 11th',
 'education_ 12th',
 'education_ 1st-4th',
 'education_ 5th-6th',
 'education_ 7th-8th',
 'education_ 9th',
 'education_ Assoc-acdm',
 'education_ Assoc-voc',
 'education_ Bachelors',
 'education_ Doctorate',
 'education_ HS-grad',
 'education_ Masters',
 'education_ Preschool',
 'education_ Prof-school',
 'education_ Some-college',
 'marital-status_ Divorced',
 'marital-status_ Married-AF-spouse',
 'marital-status_ Married-civ-spouse',
 'marital-status_ Married-spouse-absent',
 'marital-status_ Never-married',
 'marital-status_ Separated',
 'marital-status_ Widowed',
 'occupation_ ?',
 'occupation_ Adm-clerical',
 'occupation_ Ar

In [68]:
data.replace(['Divorced','Married-AF-spouse','Married-civ-spouse','Married-spouse-absent','Never-married','Separated','Widowed'],['not married','married','married','married','not married','not married','not married'], inplace =True)

In [69]:
for col in category_col:
    b, c =np.unique(data[col],return_inverse=True)
    data[col]= c

In [70]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In [71]:
col_names =data.columns
param=[]
correlation=[]
abs_corr=[]
for c in col_names:
    if c != "income":
        if len(data[c].unique()) <= 2:
            corr = spearmanr(data['income'],data[c])[0]
        else:
            corr = pointbiserialr(data['income'],data[c])[0]
        param.append(c)
        correlation.append(corr)
        abs_corr.append(abs(corr))
param_df=pd.DataFrame({'correlation' :correlation, 'parameter':param,'abs_corr':abs_corr})
param_df=param_df.sort_values(by=['abs_corr'],ascending=False)
param_df=param_df.set_index('parameter')
param_df

,correlation,abs_corr
parameter,,
education-num,0.335154,0.335154
relationship,-0.250918,0.250918
age,0.234037,0.234037
hours-per-week,0.229689,0.229689
capital-gain,0.223329,0.223329
sex,0.215980,0.215980
marital-status,-0.199307,0.199307
capital-loss,0.150526,0.150526
education,0.079317,0.079317


In [72]:
best_features=param_df.index[0:4].values
print('Best features:\t',best_features)

Best features:	 ['education-num' 'relationship' 'age' 'hours-per-week']


In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_final,income,test_size =0.2, random_state =0)
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 26048 samples.
Testing set has 6513 samples.


In [75]:
TP =np.sum(income)
FP =income.count() - TP
TN = 0
FN = 0
accuracy= TP/(TP+FP+TN+FN)
recall=TP/(TP+FN)
precision=TP/(TP+FP)
beta=0.5
fscore=(1+beta**2)*((precision*recall)/((beta**2)*precision+recall))
print("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

Naive Predictor: [Accuracy score: nan, F-score: nan]


In [77]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
def train_predict(learner, sample_size,X_train,y_train,X_test,y_test):
    results={}
    learner=learner.fit(X_train[:sample_size], y_train[:sample_size])
    predictions_test=learner.predict(X_test)
    predictions_train=learner.predict(X_train[:300])
    results['acc_train']=accuracy_score(y_train[:300],predictions_train)
    results['acc_test']=accuracy_score(y_test,predictions_test)
    results['f_train']=fbeta_score(y_train[:300],predictions_train,beta=0.5)
    results['f_test']=fbeta_score(y_test,predictions_test,beta=0.5)
    print("{} trained on {} samples.".format(learner.__class__.__name__,sample_size))
    return results
random_state =42
clf_A=RandomForestClassifier(random_state=random_state)
clf_B=GaussianNB()
samples_100=len(y_train)
samples_10=int(len(y_train)/10)
samples_1=int(len(y_train)/100)
results={}
for clf in [clf_A, clf_B]:
    clf_name=clf.__class__.__name__
    results[clf_name]={}
    for i, samples in enumerate([samples_1,samples_10,samples_100]):
        results[clf_name][i]=\
        train_predict(clf,samples,X_train,y_train,X_test,y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(solver='newton-cg')
logistic.fit(X_train,y_train)